In [2]:
def crawl_detail_info(detail_url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(detail_url)
#     time.sleep(1)
    html_txt = driver.page_source
    soup = BeautifulSoup(html_txt, 'html.parser')

   
    info_dict = {}
    
    try:
        name_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.wrap-detail-spot.js-object-fit > div.group-info > div.info-price.box.mode-basic > div.title-area > h3")
        info_dict["차종"] = name_tag.text.strip()
    except:
        info_dict["차종"] = None
    try:
        price_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.wrap-detail-spot.js-object-fit > div.group-info > div.info-price.box.mode-basic > div.price-area > span")
        info_dict["가격"] = price_tag.text.strip()
    except:
        info_dict["가격"] = None
    
    
    try:
        tbody_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.detail-section.mode-half > div:nth-child(1) > div.info-basic > div > table > tbody")

        # 각 tr 태그에 대해 반복하여 정보를 추출하여 딕셔너리에 저장하고, 리스트에 추가
        for tr_tag in tbody_tag.find_all('tr'):
            th_tags = tr_tag.find_all('th')
            td_tags = tr_tag.find_all('td')

            for th_tag, td_tag in zip(th_tags, td_tags):
                # 각 th 태그와 td 태그의 텍스트를 키와 값으로 저장
                key = th_tag.text.strip()
                value = td_tag.text.strip()
                if key not in ["배기량","연식","주행거리","변속기","색상","연료"]:
                    break
                info_dict[key] = value        
    except:
        
        pass
#     - 소유자변경유무
#     - 전손유무
#     - 침수유무

    try:
        flt_tags = soup.select("#bobaeConent > div.component.page-detail > div.container-detail > div:nth-child(5) > div > div.info-insurance.mode-personal > div.insurance-cont > dl.flt > dd")
        if len(flt_tags) >= 2:
            info_dict["소유자 이전 횟수"] = flt_tags[0].text.strip().split("/")[-1]
            info_dict["전손유무"] = flt_tags[1].text.strip().split("/")[0]
            info_dict["침수유무"] = flt_tags[1].text.strip().split("/")[2] if len(flt_tags[1].text.strip().split("/")) > 2 else None
    except:
        None
        
# - 옵션
#     - 썬루프
#     - 내비게이션
#     - 스마트키
#     - 오토라이트
#     - 주차감지센서
#     - 가죽시트
#     - 열선시트
#     - 통풍시트
#     option_selector = ".detail-option-container"
    
#     option_containers = soup.select(option_selector)
    
    # 옵션들을 찾기 위한 선택자
    option_selector = ".option-list-container"
    
    # 옵션들을 모두 선택
    option_containers = soup.select(option_selector)
    try:
        for option_container in option_containers:
        # 대분류에서 모든 옵션을 선택
            options = option_container.select("span > label > button")

            # 각 옵션에 대해 순회하면서 필요한 정보 가져오기
            for option in options:
                # 버튼 태그에서 텍스트를 가져옴
                button_text = option.text.strip()

                # 버튼 텍스트가 리스트에 없는 경우 건너뜀
                if button_text not in ["선루프", "네비게이션(순정)","네비게이션(비순정)", "스마트키", "LED헤드램프","어댑티드헤드램프", "후방센서", "가죽시트", "열선시트(앞좌석)","열선시트(뒷자석)", "통풍시트"]:
                    continue

                # 버튼 태그의 부모 li 태그에서 checked 속성을 찾음
                checked_attribute = option.find_parent("li").find("input", {"checked": True})

                # checked 속성 존재하면 1
                if checked_attribute:
                    value = 1
                else:
                    value = 0

                # 대분류와 옵션 텍스트를 조합하여 키로, 값으로 설정하여 사전에 저장
                info_dict[button_text] = value
    except:
        pass
    
    return info_dict


In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

base_url = "https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K"

# 각 페이지마다의 상세 페이지 링크를 저장
detail_links = []

# 페이지 이동
for page_num in range(1, 28):  # 숫자조절하면서 페이지반복 1페이지부터 706페이지까지 반복해야함
    url = f"{base_url}&page={page_num}&order=S11&view_size=20"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 각 페이지에서 상세 페이지 링크 추출
    car_elements = soup.select('#listCont > div.wrap-thumb-list > ul > li > div > div.mode-cell.title > p.tit > a')
    for car_element in car_elements:
        detail_link = car_element['href']
        detail_links.append(detail_link)
    
    print(page_num)
# 상세 정보를 저장할 리스트 초기화
car_info_list = []

#print(detail_links)

# time.sleep(1)

# 각 상세 페이지에 접속하여 정보 크롤링
for detail_link in detail_links:
    detail_link = f"https://www.bobaedream.co.kr{detail_link}" 
    detail_info = crawl_detail_info(detail_link)
    car_info_list.append(detail_info)
#     print(car_info_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


KeyboardInterrupt: 

In [4]:
# 실행중간에 멈추고 확인할때
for car_info in car_info_list:
    print(car_info)
print(len(car_info_list))

{'차종': '기아 더 K9 3.8 AWD  플래티넘Ⅱ', '가격': '3,450만원', '연식': '2019.04', '배기량': '3,778 cc (315마력)', '주행거리': '48,830 km', '색상': '흰색', '변속기': '자동', '연료': '가솔린', '선루프': 1, 'LED헤드램프': 1, '어댑티드헤드램프': 1, '가죽시트': 1, '열선시트(앞좌석)': 1, '통풍시트': 1, '후방센서': 1, '스마트키': 1, '네비게이션(순정)': 1, '네비게이션(비순정)': 0}
{'차종': '기아 더 K9 3.8  플래티넘', '가격': '2,500만원', '연식': '2019.01', '배기량': '3,778 cc (315마력)', '주행거리': '105,000 km', '색상': '검정색', '변속기': '자동', '연료': '가솔린', '소유자 이전 횟수': ' 1회', '전손유무': '전손: 0 ', '침수유무': ' 침수분손 : 0 '}
{'차종': '현대 아반떼MD 1.6 M16 프리미엄', '가격': '399만원', '연식': '2011.02', '배기량': '1,591 cc (140마력)', '주행거리': '113,000 km', '색상': '회색', '변속기': '자동', '연료': '가솔린', '선루프': 0, 'LED헤드램프': 0, '어댑티드헤드램프': 0, '가죽시트': 1, '열선시트(앞좌석)': 1, '통풍시트': 0, '후방센서': 1, '스마트키': 1, '네비게이션(순정)': 0, '네비게이션(비순정)': 0}
{'차종': '기아 카니발 4세대 2.2 디젤 7인승 하이리무진  시그니처', '가격': '4,350만원', '연식': '2021.11', '배기량': '2,151 cc (202마력)', '주행거리': '80,848 km', '색상': '흰색', '변속기': '자동', '연료': '디젤', '선루프': 0, 'LED헤드램프': 1, '어댑티드헤드램프': 0, '가죽시트': 1, '열선시트(앞좌석

In [5]:
import pandas as pd

car_info_df = pd.DataFrame(car_info_list)
car_info_df

,차종,가격,연식,배기량,주행거리,색상,변속기,연료,선루프,LED헤드램프,...,가죽시트,열선시트(앞좌석),통풍시트,후방센서,스마트키,네비게이션(순정),네비게이션(비순정),소유자 이전 횟수,전손유무,침수유무
0,기아 더 K9 3.8 AWD 플래티넘Ⅱ,"3,450만원",2019.04,"3,778 cc (315마력)","48,830 km",흰색,자동,가솔린,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
1,기아 더 K9 3.8 플래티넘,"2,500만원",2019.01,"3,778 cc (315마력)","105,000 km",검정색,자동,가솔린,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1회,전손: 0,침수분손 : 0
2,현대 아반떼MD 1.6 M16 프리미엄,399만원,2011.02,"1,591 cc (140마력)","113,000 km",회색,자동,가솔린,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN
3,기아 카니발 4세대 2.2 디젤 7인승 하이리무진 시그니처,"4,350만원",2021.11,"2,151 cc (202마력)","80,848 km",흰색,자동,디젤,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
4,현대 그랜저IG 2.4 프리미엄,"2,090만원",2018.02,"2,359 cc (190마력)","18,723 km",흰색,자동,가솔린,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
5,KG모빌리티 티볼리 1.6 가솔린 2WD VX,940만원,2016.06,"1,597 cc (126마력)","98,172 km",흰색,자동,가솔린,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN
6,기아 올 뉴 K3 1.6 트렌디,"1,130만원",2019.09 (20년형),"1,598 cc (123마력)","99,965 km",검정색,자동,가솔린,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN


In [7]:
filename = 'car_data.csv'
car_info_df.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"'{filename}' 파일이 성공적으로 저장되었습니다.")

'car_data.csv' 파일이 성공적으로 저장되었습니다.


In [ ]:
for detail_link in detail_links:
    detail_link = f"https://www.bobaedream.co.kr{detail_link}" 
    detail_info = crawl_detail_info(detail_link)
    car_info_list.append(detail_info)

In [63]:
del(car_info_list)

In [64]:
car_info_list = []

In [79]:
len(detail_links)

210

In [80]:
len(set(detail_links))

210

In [5]:
detail_links

['/cyber/CyberCar_view.php?no=771012&gubun=K',
 '/cyber/CyberCar_view.php?no=770831&gubun=K',
 '/cyber/CyberCar_view.php?no=770812&gubun=K',
 '/cyber/CyberCar_view.php?no=769118&gubun=K',
 '/cyber/CyberCar_view.php?no=763555&gubun=K',
 '/cyber/CyberCar_view.php?no=771014&gubun=K',
 '/cyber/CyberCar_view.php?no=770993&gubun=K',
 '/cyber/CyberCar_view.php?no=771002&gubun=K',
 '/cyber/CyberCar_view.php?no=770953&gubun=K',
 '/cyber/CyberCar_view.php?no=770981&gubun=K',
 '/cyber/CyberCar_view.php?no=770986&gubun=K',
 '/cyber/CyberCar_view.php?no=771011&gubun=K',
 '/cyber/CyberCar_view.php?no=771010&gubun=K',
 '/cyber/CyberCar_view.php?no=771004&gubun=K',
 '/cyber/CyberCar_view.php?no=770997&gubun=K',
 '/cyber/CyberCar_view.php?no=770823&gubun=K',
 '/cyber/CyberCar_view.php?no=770909&gubun=K',
 '/cyber/CyberCar_view.php?no=770951&gubun=K',
 '/cyber/CyberCar_view.php?no=770973&gubun=K',
 '/cyber/CyberCar_view.php?no=770947&gubun=K',
 '/cyber/CyberCar_view.php?no=770979&gubun=K',
 '/cyber/Cybe

In [77]:
len(car_info_list)

326